In [1]:
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

import rasterio
from rasterio.enums import Resampling
from rasterio.merge import merge
from rasterio.plot import show
from rasterio.fill import fillnodata
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.plot import show

In [2]:
# WD = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/'
WD_INPUT_25m = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Inputmaps_25m/'
WD_INPUT_8m = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Inputmaps_8m/'

dem_path_25m = 'DEM_clipped_25m.asc'

SimDomain_path_8m = 'SimDomain_8m.asc'

WD_RESAMPLED = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Working_folder/'
WD_INPUT = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Inputmaps_8m/'

NO_DATA = -9999

## Creating an empty grid with 8 grid cells

In [3]:
# The downscale factor is 5 because the used DEM is 25 m and we want to go 8.33 m (25/3 = 8.33)
downscale_factor = 3

# with rasterio.open(f'{WD_INPUT_25m}{dem_path_25m}') as dst:
with rasterio.open(f'{WD_INPUT_25m}{dem_path_25m}') as dst:

    # resample data to target shape
    data = dst.read(
        out_shape=(
            dst.count,
            int(dst.height * downscale_factor),
            int(dst.width * downscale_factor)
        ),
        resampling=Resampling.bilinear
    )

    # scale image transform
    transform = dst.transform * dst.transform.scale(
        (dst.width / data.shape[-1]),
        (dst.height / data.shape[-2])
    )
    
with rasterio.open(
    f'{WD_INPUT_8m}{SimDomain_path_8m}',
    'w',
    driver= 'AAIGrid',
    dtype = 'float32',
    crs = dst.crs,
    transform = transform,
    height = data.shape[1],
    width = data.shape[2],
    count = 1,
    nodata = NO_DATA) as dst:
    dst.write(data * 0)         # times zero to create empty grid

## Setting the project characteristics to be used from the simulation domain

In [4]:
with rasterio.open(f'{WD_INPUT_8m}{SimDomain_path_8m}') as proj_outline:
    PROJECT_CRS = proj_outline.crs
    PROJECT_TRANSFORM = proj_outline.transform
    PROJECT_WIDTH = proj_outline.width
    PROJECT_HEIGHT = proj_outline.height
    PROJECT_SHAPE = proj_outline.shape
    PROJECT_BOUNDS = proj_outline.bounds
proj_outline.close()

### Function for resampling

In [5]:
def Resample(Input_TIF, Output_ASC):

    with rasterio.open(Input_TIF, 'r') as src_proj:
        with rasterio.open(Output_ASC, 
                           'w', 
                           driver = 'AAIGrid', 
                           dtype = 'float32',
                            crs = PROJECT_CRS,
                            transform = PROJECT_TRANSFORM,
                            height = PROJECT_HEIGHT,
                            width = PROJECT_WIDTH,
                            count = 1,
                            nodata = NO_DATA) as dst_proj:
                                                reproject(source = rasterio.band(src_proj, 1),
                                                destination = rasterio.band(dst_proj, 1),
                                                src_transform = src_proj.transform,
                                                src_crs = src_proj.crs,
                                                dst_transform = PROJECT_TRANSFORM,
                                                dst_crs = PROJECT_CRS,
                                                resampling = Resampling.average,
                                                bounds = PROJECT_BOUNDS)   

Input_Wallonie = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Belgium/Wallonie/RELIEF_WALLONIE_MNT_2013_2014.tif'
Input_Flanders = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Belgium/Flanders/GeoTIFF/DHMVIIDTMRAS1m_k34.tif'
Input_Netherlands = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Netherlands/NL_Mergedfile.tif'
Input_Germany = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Germany/GER_mergedfile.tif'

Output_Wallonie = f'{WD_RESAMPLED}Wallonie_Resamp.asc'
Output_Flanders = f'{WD_RESAMPLED}Flanders_Resamp.asc'
Output_Netherlands = f'{WD_RESAMPLED}Netherlands_Resamp.asc'
Output_Germany = f'{WD_RESAMPLED}Germany_Resamp.asc'

In [6]:
Resample(Input_Wallonie, Output_Wallonie)
Resample(Input_Flanders, Output_Flanders)
Resample(Input_Netherlands, Output_Netherlands)
Resample(Input_Germany, Output_Germany)

### Adjusting all rasters to the same reference height as the dutch DEM

The DEM data of each dem is cut out at the place where it overlaps with the DEM of Netherlands. The mean difference in this region is calculated and then summed with whole raster to adjusted it to the reference height of the raster of the Netherlands

IMPORTANT
NO_DATA must be a large negative number for this to work. Preferably this is -9999 (which it should be
anyway at this point to be able to run in the RIM2D model)

In [7]:
def Adjusting_Ras_Ref(ref_ras, adjust_ras, output_adjust_ras):
    with rasterio.open(ref_ras) as dst_ref:
        with rasterio.open(adjust_ras) as dst_adjust:
            arr_ref = dst_ref.read(1)
            arr_adjust = dst_adjust.read(1)
            
            ref_plus_adjust = arr_ref + arr_adjust
            
            ref_clip = np.where(ref_plus_adjust > 0, arr_ref, NO_DATA)
            adjust_clip = np.where(ref_plus_adjust > 0, arr_adjust, -NO_DATA)
            difference_ref_adjust = ref_clip - adjust_clip
            
            adjustment_number = np.mean(difference_ref_adjust[difference_ref_adjust > NO_DATA])
            print('adjustment_number : ' + str(adjustment_number))
            
            new_arr_adjust = np.where(arr_adjust > NO_DATA, (arr_adjust + adjustment_number), NO_DATA)
            
            kwargs = dst_adjust.profile
            kwargs.update(driver = 'AAIGrid')
            
    with rasterio.open(output_adjust_ras, 'w', **kwargs) as dst:
        dst.write(new_arr_adjust, indexes = 1)

# Filenames
# Input
Neth_Ref = 'Netherlands_Resamp.asc' 

# Output 
Wal_Adj = 'Wallonie_Adjusted.asc'
Flan_Adj = 'Flanders_Adjusted.asc'
Ger_Adj = 'Germany_Adjusted.asc'

# Paths
# Input
Neth_Ref_Path = f'{WD_RESAMPLED}{Neth_Ref}'
# Output
Out_Wal_Adj_Path = f'{WD_RESAMPLED}{Wal_Adj}'
Out_Flan_Adj_Path = f'{WD_RESAMPLED}{Flan_Adj}'
Out_Ger_Adj_Path = f'{WD_RESAMPLED}{Ger_Adj}'
       
# # Running funtion
Adjusting_Ras_Ref(Neth_Ref_Path, Output_Wallonie, Out_Wal_Adj_Path)
Adjusting_Ras_Ref(Neth_Ref_Path, Output_Flanders, Out_Flan_Adj_Path)
Adjusting_Ras_Ref(Neth_Ref_Path, Output_Germany, Out_Ger_Adj_Path)

adjustment_number : -2.421606
adjustment_number : -2.3338041
adjustment_number : -0.058213618


### Merging DEMs

In [8]:
Paths = [Neth_Ref_Path, Out_Wal_Adj_Path, Out_Flan_Adj_Path, Out_Ger_Adj_Path]

files_to_merge = []

for i in Paths:
    dst = rasterio.open(i)
    files_to_merge.append(dst)

In [9]:
mosaic, transform = merge(files_to_merge)

# show(mosaic, cmap='terrain')

kwargs = dst.meta.copy()
kwargs.update({'driver': 'GTiff',
                  'height': mosaic.shape[1],
                  'width': mosaic.shape[2],
                  'transform': transform,
                  'crs': PROJECT_CRS
                  }
                 )

with rasterio.open(f'{WD_RESAMPLED}Merged_Open_Gaps.tif', 
                   'w',
                  **kwargs) as dst:
    dst.write(mosaic)

### Filling gaps (in Dutch DEM)

In [11]:
def FillGaps(WithGaps, WithoutGaps): 

    with rasterio.open(WithGaps) as src:
        kwargs = src.profile
        arr = src.read(1)
        arr_filled = fillnodata(arr, mask=src.read_masks(1), max_search_distance=35, smoothing_iterations=0)
        
        kwargs.update(driver = 'AAIGrid')
        
    with rasterio.open(WithoutGaps, 'w', **kwargs) as dest:
        dest.write_band(1, arr_filled)

        
# # This is to close gaps in the Netherlands DEM after merging the different DEMS. One of the FillGaps below works
# WithGaps = f'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Working_folder/Merged_Open_Gaps.tif'
# WithoutGaps = f'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Merged_Closed_Gaps.tif'
# FillGaps(f'{WD_RESAMPLED}Merged_Open_Gaps.tif', f'{WD_RESAMPLED}Merged_Closed_Gaps.tif')
# FillGaps(Output_Netherlands, WithoutGapsNL)

# # This is to fill the gaps in the Netherlands DEM before merging        
# WithGaps = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Netherlands/NL_Mergedfile.tif'
# WithoutGaps = 'C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Netherlands/NL_Mergedfile_FilledGaps.tif'               
# FillGaps(WithGaps, WithoutGaps)

# This i to fill gaps in the mergad raster
FillGaps(f'{WD_RESAMPLED}Merged_Open_Gaps.tif', f'{WD_INPUT}Merged_Closed_Gaps_8m.asc') # Saving this file in input because need it there

### Old stuff

In [ ]:
########
# Ned_plus_Wal = arr_Ned_OpenGaps+arr_Wal
# Ned_clip = np.where(Ned_plus_Wal>0, arr_Ned_OpenGaps, NO_DATA) 
# Wal_clip = np.where(Ned_plus_Wal>0, arr_Wal, -NO_DATA) # Negative NO_DATA so when extracting Wal_Clip from Ned_Clip below, the Nodata value becomes -19998 and not 0.

# Difference_Ned_Wal = Ned_clip - Wal_clip
# AdjustmentNumber = np.mean(Difference_Ned_Wal[Difference_Ned_Wal > NO_DATA])
# print(AdjustmentNumber)

In [56]:
# # The filled gaps only worked with .tif files, so this is to save the .tif file as .asc file
# with rasterio.open('C:/Users/siepb/Documents/Studie_Hydrology/ThesisGeul/Raw_DEM/DEM_HighRes/Working_folderNetherlands_Resamp_5mresamp.tif') as dst:
#     kwargs = dst.profile
#     kwargs.update(driver = 'AAIGrid')
    
#     with rasterio.open(Output_Netherlands, 'w', **kwargs) as new_dst:
#         new_dst.write(dst.read(1), indexes = 1)

In [ ]:
# # Other way of getting pats
# search_crit = '*Resamp.asc' 
# search_path = os.path.join(WD_RESAMPLED, search_crit)
# Paths = glob.glob(search_path)